In [1]:
using Pkg;
Pkg.activate(".")
Pkg.add("BenchmarkTools")
Pkg.add("DataFrames")

using BenchmarkTools
using DataFrames

  Activating project at `C:\Users\Janelle`
    Updating registry at `C:\Users\Janelle\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Janelle\Project.toml`
  No Changes to `C:\Users\Janelle\Manifest.toml`
   Resolving package versions...
   Installed StringManipulation ─ v0.3.0
   Installed Crayons ──────────── v4.1.1
   Installed PooledArrays ─────── v1.4.2
   Installed InvertedIndices ──── v1.2.0
   Installed DataFrames ───────── v1.4.4
   Installed PrettyTables ─────── v2.2.2
    Updating `C:\Users\Janelle\Project.toml`
  [a93c6f00] + DataFrames v1.4.4
    Updating `C:\Users\Janelle\Manifest.toml`
  [a8cc5b0e] + Crayons v4.1.1
  [a93c6f00] + DataFrames v1.4.4
  [41ab1584] + InvertedIndices v1.2.0
  [2dfb63ee] + PooledArrays v1.4.2
  [08abe8d2] + PrettyTables v2.2.2
  [892a3eda] + StringManipulation v0.3.0
Precompiling project...
  ✓ InvertedIndices
  ✓ PooledArrays
  ✓ Crayon

# KR1: Benchmarked at least two(2) different implementation of the same function or process (e.g. raising each element of an array to some power p, random array may be used) that utilizes some parameter that can be considered a constant or declared globally. Typical methods: (1) Global variable, (2) Constant global variable, and (3) Named parameter variable.

In [2]:
#Using Globals

p = 2 

function pow_array(x::Vector{Float64}) 
   s = 0.0 
   for y in x 
      s = s + y^p 
   end 
   return s 
end 

pow_array (generic function with 1 method)

In [3]:
t=rand(100000)

100000-element Vector{Float64}:
 0.8924325943637382
 0.4954125889978741
 0.3583797283395247
 0.3120243970171511
 0.600737059344133
 0.6367340808758559
 0.8253525872564944
 0.519969185557728
 0.19632320239664836
 0.7304232493669719
 0.0851132655482596
 0.059412920406394454
 0.600118231385412
 ⋮
 0.621375717035871
 0.26808398453385573
 0.1648205219326374
 0.6061091769487942
 0.4193399083903083
 0.28597924856889023
 0.11738786210907881
 0.0831056565296232
 0.40416195028093005
 0.2319990705265671
 0.5456395148377817
 0.3459858843386585

In [4]:
@btime pow_array($t)

  5.720 ms (300000 allocations: 4.58 MiB)


33462.11918685212

In [5]:
@code_warntype pow_array(t)

MethodInstance for pow_array(::Vector{Float64})
  from pow_array(x::Vector{Float64}) in Main at In[2]:5
Arguments
  #self#::Core.Const(pow_array)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  y::Float64
Body::Any
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



The compiler has been unable to infer the type of result when working with the global variable ("Any")

In [7]:
#Using Constant globals

const p1 = 2 
function pow_array1(x::Vector{Float64}) 
    s = 0.0 
    for y in x 
        s = s + y^p1
    end 
    return s 
end 

pow_array1 (generic function with 1 method)

In [8]:
@btime pow_array1($t)

  352.600 μs (0 allocations: 0 bytes)


33462.11918685212

Using a global constant gave us better performance. 

#  KR2: Replicated the naive implementation of the polynomial in the textbook.

In [9]:
function poly_naive(x, a...) 
   p=zero(x) 
   for i = 1:length(a) 
      p = p + a[i] *  x^(i-1) 
   end 
   return p 
end

poly_naive (generic function with 1 method)

In [10]:
f_naive(x) = poly_naive(x, 1,2,3,4,5,6,7,8,9)

f_naive (generic function with 1 method)

In [20]:
x=3.5
mark1= @benchmark f_naive($x)


BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  32.460 ns … 405.847 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     35.484 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   37.614 ns ±   6.466 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▂ █▄                                                        
  ▁██▃██▇▃▄▅▃▂▂▂▂▇▄▂▅▃▂▂▆▂▂▂▂▂▁▁▁▁▁▂▁▁▁▂▃▃▁▁▁▁▂▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁ ▂
  32.5 ns         Histogram: frequency by time         54.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# KR3: Replicated the naive implementation of the Horner's method for the same polynomial.

In [12]:
function poly_horner(x, a...) 
   b=zero(x) 
   for i = length(a):-1:1 
      b = a[i] + b * x 
   end 
   return b 
end 

poly_horner (generic function with 1 method)

In [13]:
f_horner(x) = poly_horner(x, 1,2,3,4,5,6,7,8,9)

f_horner (generic function with 1 method)

In [21]:
mark2= @benchmark f_horner($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  5.400 ns … 95.800 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.600 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.963 ns ±  1.529 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇▇█▆▅▄ ▃▂▃▂▄                                  ▂▁▁ ▁ ▁▁▁    ▂
  ██████▁█████▁█▆▆▅▆▇▁▇▆▇▆▇▁▆▆▆▆▆▆▁▆▆▆▆▄▁▆▅▄▅▆█▁█████▁████▇▅ █
  5.4 ns       Histogram: log(frequency) by time     10.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# **KR4:** Replicated the macro implementation of the Horner's method of the same polynomial.

In [17]:
macro horner(x, p...)
   ex = esc(p[end])
   for i = length(p)-1:-1:1
      ex = :(muladd(t, $ex, $(esc(p[i]))))
   end
   Expr(:block, :(t = $(esc(x))), ex)
end




@horner (macro with 1 method)

In [18]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_horner_macro (generic function with 1 method)

In [22]:
mark3= @benchmark f_horner_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  3.500 ns … 19.900 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.700 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.823 ns ±  0.675 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▇ █ ▆ ▂                                                   ▂
  ▃█▁█▁█▁█▁▁▄▁▄▁▄▁▃▁▃▄▁▁▁▁▁▁▁▃▄▁▇▁█▁▇▁▆▁▆▆▁▇▁▇▁█▁██▁█▁█▁█▁█▅ █
  3.5 ns       Histogram: log(frequency) by time      6.7 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# **KR5:** Table showing how many _minutes_ will the function evaluations in both KR3 and KR4 be reduced if KR2 requires 24hours of runtime.

In [23]:
speedup1 = median(mark1.times) / median(mark2.times)

table = DataFrame("Method"=>["Naive","Horner"],"Speedup" => [1.0, speedup1]);

print(table)

2×2 DataFrame
 Row │ Method  Speedup 
     │ String  Float64 
─────┼─────────────────
   1 │ Naive   1.0
   2 │ Horner  6.33641

In [24]:
speedup2 = median(mark1.times) / median(mark3.times)

push!(table, ["Macro", speedup2]);

print(table)

3×2 DataFrame
 Row │ Method  Speedup 
     │ String  Float64 
─────┼─────────────────
   1 │ Naive   1.0
   2 │ Horner  6.33641
   3 │ Macro   9.59024

In [25]:
for r in eachrow(table)
    println("$(24*60/r.Speedup) mins for $(r.Method) method.")
end

1440.0 mins for Naive method.
227.2581818181818 mins for Horner method.
150.1527272727273 mins for Macro method.


In [26]:
transform!(table,:Speedup=>ByRow(x->24*60/x)=>:"Time(mins)")
print(table)

3×3 DataFrame
 Row │ Method  Speedup  Time(mins) 
     │ String  Float64  Float64    
─────┼─────────────────────────────
   1 │ Naive   1.0        1440.0
   2 │ Horner  6.33641     227.258
   3 │ Macro   9.59024     150.153